In [ ]:
import tensorflow as tf
import numpy as np

import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

# Import Plotting Libararies
import seaborn as sns
import matplotlib.pyplot as plt

# Import Data Preprocessing Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Machine Learning Models
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Model Evaluation Libraries
from sklearn.metrics import classification_report, confusion_matrix
import sklearn.metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import preprocessing

In [ ]:
shayari = pd.read_csv("dataset.csv", encoding='utf-16', header=None, sep='\t', names=['text','labels'])

In [ ]:
from tokenizers import ByteLevelBPETokenizer

# Initialize the tokenizer
tokenizer = ByteLevelBPETokenizer()

# Tokenize the text data
tokenizer.train_from_iterator(shayari['text'], vocab_size=50000)


In [ ]:
# Encode the sequences
encoded_sequences = [tokenizer.encode(text).ids for text in shayari['text']]

# Prepare sequences for training
max_length = max(len(sequence) for sequence in encoded_sequences)
padded_sequences = [sequence + [0] * (max_length - len(sequence)) for sequence in encoded_sequences]


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer, Trainer, TrainingArguments

# Define the model configuration
model_config = GPT2Config(vocab_size=tokenizer.get_vocab_size(), n_positions=max_length)

# Initialize the GPT-2 model
model = GPT2LMHeadModel(config=model_config)

# Initialize the tokenizer for the GPT-2 model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

In [ ]:
!pip install transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

In [ ]:
# Initialize the GPT-2 model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Convert sequences to token IDs
tokenized_sequences = [tokenizer.encode(sequence, return_tensors='pt') for sequence in shayari['text']]

In [ ]:
# Pad sequences to the same length
max_length = max(len(sequence[0]) for sequence in tokenized_sequences)
padded_sequences = [torch.cat((sequence, torch.zeros(1, max_length - len(sequence), dtype=torch.long)), dim=1) for sequence in tokenized_sequences]

In [ ]:
# Create DataLoader for training
dataset = TensorDataset(*padded_sequences)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# Define training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 3

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        # Unpack the batch
        batch = batch[0]  # Since DataLoader returns a list of tensors, we access the first element to get the tensor
        inputs, labels = batch[:, :-1], batch[:, 1:]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs.logits.permute(0, 2, 1), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save the trained model
model.save_pretrained("./output")

Epoch 2/3, Loss: 0.9777524471282959
Epoch 3/3, Loss: 0.7333675622940063


In [ ]:
from transformers import GPT2LMHeadModel

# Load the trained model
model = GPT2LMHeadModel.from_pretrained("./output")

In [ ]:
# Example prompt
prompt_text = "محبت"

In [ ]:
# Load the tokenizer from the pre-trained model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

# Generate poetry based on the prompt
generated_poetry = model.generate(
    input_ids=tokenizer.encode(prompt_text, return_tensors='pt'),
    max_length=100,  # Adjust the maximum length of the generated poetry
    num_return_sequences=1,  # Number of different poems to generate
    temperature=0.7,  # Adjust the temperature to control the randomness of the generation
    top_k=50,  # Adjust the top-k sampling parameter
    top_p=0.95,  # Adjust the top-p sampling parameter
    pad_token_id=tokenizer.eos_token_id,  # Use the end-of-sequence token as the padding token
    bos_token_id=tokenizer.bos_token_id,  # Use the beginning-of-sequence token
    eos_token_id=tokenizer.eos_token_id,  # Use the end-of-sequence token
    do_sample=True,
)

# Decode the generated poetry from token IDs to text
decoded_poetry = tokenizer.decode(generated_poetry[0], skip_special_tokens=True)
print(decoded_poetry)


محبت کی مشابہت، تو نے کیا کیا کیا دل کی دھڑکنوں کو، یہ کیا معلوم کیا ��ǽ��


In [ ]:
# Generate poetry based on the seed word with adjusted temperature
generated_poetry = model.generate(
    input_ids=tokenizer.encode(seed_word, return_tensors='pt'),
    max_length=50,  # Adjust the maximum length of the generated poetry
    num_return_sequences=2,  # Number of different poems to generate
    temperature=0.9,  # Adjust the temperature to control the randomness of the generation
    top_k=50,  # Adjust the top-k sampling parameter
    top_p=0.95,  # Adjust the top-p sampling parameter
    pad_token_id=tokenizer.eos_token_id,  # Use the end-of-sequence token as the padding token
    bos_token_id=tokenizer.bos_token_id,  # Use the beginning-of-sequence token
    eos_token_id=tokenizer.eos_token_id,  # Use the end-of-sequence token
    do_sample=True,
)

# Decode and print the generated poetry
decoded_poetry = [tokenizer.decode(poetry, skip_special_tokens=True) for poetry in generated_poetry]
for i, poetry in enumerate(decoded_poetry, start=1):
    print(f"Line {i}: {poetry}")


Line 1: دلنا ہراہت سرار ہرو سرنت عہت سرنا علہت سرو عل
Line 2: دل ہیسر مونینین عہر حہر بن سہر سران سرتناین


In [ ]:
from tokenizers import ByteLevelBPETokenizer
import os

# Step 1: Ensure Proper Encoding
# Convert dataset text file to UTF-8 format if needed
input_file = "/content/textdataset.txt"
output_file = "/content/textdataset_utf8.txt"

with open(input_file, "r", encoding="utf-16") as f:
    text = f.read()

with open(output_file, "w", encoding="utf-8") as f:
    f.write(text)

# Step 2: Train the Tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=output_file, vocab_size=52000, min_frequency=2)

# Step 3: Save the Tokenizer
tokenizer_dir = "tokenizer_directory"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save_model(tokenizer_dir)


['tokenizer_directory/vocab.json', 'tokenizer_directory/merges.txt']

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("tokenizer_directory")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")  # or "gpt2-large" or "gpt2-xl" depending on your resource constraints


In [ ]:
# Fine-tune the model on your poetry dataset
# (This code assumes you've already formatted your data into a suitable format for training)
# (You may need to adapt this code based on your specific dataset and training process)
model.train()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
model.save_pretrained("fine_tuned_model_directory")

In [ ]:
from transformers import GPT2LMHeadModel

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("fine_tuned_model_directory")


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("fine_tuned_model_directory")

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("tokenizer_directory")

# Set the model to evaluation mode
model.eval()

# Generate poetry
prompt = "سنا ہے"  # You can provide any prompt you like
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate poetry based on the prompt
output = model.generate(input_ids, max_length=100, temperature=1.0, pad_token_id=tokenizer.eos_token_id, num_beams=5, num_return_sequences=3)

# Decode and print the generated poetry
for i, sample_output in enumerate(output):
    print(f"Generated Poetry {i+1}: {tokenizer.decode(sample_output, skip_special_tokens=True)}")


Generated Poetry 1: سنا ہے اشکتھیانو.

"I تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ لیےیکؤں. فI تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا
Generated Poetry 2: سنا ہے اشکتھیانو.

"I تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ لیےیکؤں. فI تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.

"I تی کیاتا خادزتجھےد
Generated Poetry 3: سنا ہے اشکتھیانو.

"I تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ لیےیکؤں. فI تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےدنہیںطر کیا ویسی تجھ.ق تی کیاتا خادزتجھےد جیتےطر کیا


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained "gpt2-urdu-smallest" model and tokenizer from TensorFlow weights
model = GPT2LMHeadModel.from_pretrained("hadidev/gpt2-urdu-smallest", from_tf=True)
tokenizer = GPT2Tokenizer.from_pretrained("hadidev/gpt2-urdu-smallest", from_tf=True)


All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set the model to evaluation mode
model.eval()

# Generate poetry
prompt = "سنا ہے"  # You can provide any prompt you like
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate poetry based on the prompt
output = model.generate(input_ids, max_length=10, temperature=1.0, pad_token_id=tokenizer.eos_token_id, num_beams=5, num_return_sequences=3)

# Decode and print the generated poetry
for i, sample_output in enumerate(output):
    print(f"Generated Poetry {i+1}: {tokenizer.decode(sample_output, skip_special_tokens=True)}")


Generated Poetry 1: سنا ہے کہ اس کے بعد اس کے پاس کوئی
Generated Poetry 2: سنا ہے کہ اس کے بعد اس نے اس کی
Generated Poetry 3: سنا ہے کہ اس کے بعد اس نے اس کے
